In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import openpyxl

In [ ]:
# Function to clean data by removing columns with 'Unnamed' in the name
def clean_data(df):
    """
    This function takes a DataFrame as input, removes columns with 'Unnamed' in the column name,
    and returns the cleaned DataFrame.
    """
    df_cleaned = df.loc[:, ~df.columns.str.contains('Unnamed')]
    return df_cleaned

In [ ]:
# Function to calculate baseline (first 30 values) for a given measurement
def calculate_baseline(df, measurement_column):
    """
    This function extracts the first 30 rows from a given measurement column and returns them as the baseline.
    """
    return df.loc[1:30, measurement_column]

In [ ]:
# Function to calculate the minimum value of the baseline
def calculate_min(baseline):
    """
    This function calculates the minimum value from the baseline data.
    """
    return baseline.min()

In [ ]:
# Function to calculate the maximum value for a given measurement
def calculate_max(df, measurement_column):
    """
    This function calculates the maximum value from the entire column of the specified measurement.
    """
    return df[measurement_column].max()

In [ ]:
# Function to calculate the relative change for a given max and min
def calculate_relative_change(max_value, min_value):
    """
    This function calculates the relative change as (max - min) / min.
    """
    return (max_value - min_value) / min_value

In [ ]:
# Function to read the file path from the user input
def get_file_path():
    """
    This function prompts the user to input the file path for the Excel file.
    """
    file_path = input("Please enter the full path to your Excel file (e.g., C:/path/to/your/file.xlsx): ")
    return file_path

In [ ]:
# Ask the user to input the file path for the stimulant and negative control data
file_path = get_file_path()

In [ ]:
# Read the stimulant data from the user-provided file path
df_Stimulant = pd.read_excel(file_path, sheet_name='Stimulant')

In [ ]:
# Clean the stimulant data
df_Stimulant = clean_data(df_Stimulant)

In [ ]:
# Rename the column "Time 1" to "Time"
df_Stimulant.rename(columns={"Time 1": "Time"}, inplace=True)

# Set "Time" column as index
df_Stimulant.set_index("Time", inplace=True)

In [ ]:
# Calculate baseline values for stimulant
baseline1 = calculate_baseline(df_Stimulant, "Measurement 1")
baseline2 = calculate_baseline(df_Stimulant, "Measurement 2")
baseline3 = calculate_baseline(df_Stimulant, "Measurement 3")

# Calculate min baseline values for stimulant
min_baseline1 = calculate_min(baseline1)
min_baseline2 = calculate_min(baseline2)
min_baseline3 = calculate_min(baseline3)

# Calculate max values for stimulant
max_1 = calculate_max(df_Stimulant, "Measurement 1")
max_2 = calculate_max(df_Stimulant, "Measurement 2")
max_3 = calculate_max(df_Stimulant, "Measurement 3")

# Calculate relative changes for stimulant
relative_change1 = calculate_relative_change(max_1, min_baseline1)
relative_change2 = calculate_relative_change(max_2, min_baseline2)
relative_change3 = calculate_relative_change(max_3, min_baseline3)

# Print relative change for stimulant
print("Stimulant Relative Changes:")
print(relative_change1)
print(relative_change2)
print(relative_change3)

In [ ]:
# Read the negative control data from the same user-provided file path
df_Negative = pd.read_excel(file_path, sheet_name='Negative')

In [ ]:
# Clean the negative control data
df_Negative = clean_data(df_Negative)

# Rename the column "Time 1" to "Time"
df_Negative.rename(columns={"Time 1": "Time"}, inplace=True)

# Set "Time" column as index
df_Negative.set_index("Time", inplace=True)

In [ ]:
# Calculate baseline values for negative control
baseline1_neg = calculate_baseline(df_Negative, "Measurement 1")
baseline2_neg = calculate_baseline(df_Negative, "Measurement 2")
baseline3_neg = calculate_baseline(df_Negative, "Measurement 3")

# Calculate min baseline values for negative control
min_baseline1_neg = calculate_min(baseline1_neg)
min_baseline2_neg = calculate_min(baseline2_neg)
min_baseline3_neg = calculate_min(baseline3_neg)

# Calculate max values for negative control
max_1_neg = calculate_max(df_Negative, "Measurement 1")
max_2_neg = calculate_max(df_Negative, "Measurement 2")
max_3_neg = calculate_max(df_Negative, "Measurement 3")

# Calculate relative changes for negative control
relative_change1_neg = calculate_relative_change(max_1_neg, min_baseline1_neg)
relative_change2_neg = calculate_relative_change(max_2_neg, min_baseline2_neg)
relative_change3_neg = calculate_relative_change(max_3_neg, min_baseline3_neg)

# Print relative change for negative control
print("Negative Control Relative Changes:")
print(relative_change1_neg)
print(relative_change2_neg)
print(relative_change3_neg)

In [ ]:
# Create a summary dataframe
summary_file = {
    'Relative change 1': [relative_change1, relative_change1_neg],
    'Relative change 2': [relative_change2, relative_change2_neg],
    'Relative change 3': [relative_change3, relative_change3_neg],
}

summary_df = pd.DataFrame(summary_file, index=["Stimulant", "Negative"])

# Print summary dataframe
print(summary_df)

In [ ]:
# Perform paired t-test
Stimulant = [relative_change1, relative_change2, relative_change3]
Negative = [relative_change1_neg, relative_change2_neg, relative_change3_neg]

t_stat, p_value = stats.ttest_rel(Stimulant, Negative)

# Print the results of the t-test
print(r" T-statistics: ", t_stat)
print(r" P-value : ", p_value)
if p_value < 0.5:
    print(r" There is a significant difference between Stimulant and Negative")
else:
    print(r" No significant difference between Stimulant and Negative")

In [ ]:
# Plotting the data
df_Stimulant['Measurement 1'].plot(marker='o', linestyle='-', color='b', label='Stimulant')
df_Negative['Measurement 1'].plot(marker='o', linestyle='-', color='r', label='Negative')

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Ca2+ Levels')
plt.title('Relative Ca2+ Change - Measurement 1')

# Add grid and legend
plt.legend()

# Save and show the plot
plt.savefig('measurement_1_plot.png', dpi=600)
plt.show()